In [8]:
import pandas as pd
import numpy as np

In [3]:
key = '1CZqJ-z0i2yGqFr5dTOG2YOYngKx8ngVVqsnPbja95-A'

In [157]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/' +
                 key +
                 '/export?gid=0&format=csv',
                 parse_dates=['Date'],
                 dayfirst=True)

/var/folders/d5/fs_2ycr11tsg8bmr83z_23hr0000gp/T/ipykernel_32253/4006416093.py:1: UserWarning:

Parsing dates in %Y/%m/%d %H:%M format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.



In [158]:
def h2n(s: str) -> str:
    if isinstance(s, str):
        if ':' not in s:
            return s

        h, m = s.split(':')
        m = str(round(int(m) / 60, 2)).split('.')[-1]

        return h + '.' + m
    else:
        return s

In [159]:
df = df.replace('-', np.NaN)

df['Kk'] = df['Kk'].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
df['Kk'] = pd.to_numeric(df['Kk'], errors='coerce')

df['Kg'] = df['Kg'].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
df['Kg'] = pd.to_numeric(df['Kg'], errors='coerce')

df['Hours'] = df['Hours'].apply(lambda x: h2n(x))
df['Hours'] = pd.to_numeric(df['Hours'], errors='coerce')

In [119]:
df

,Date,Type,Info,Kk,Kg,Hours
0,2024-02-15 21:00:00,Sleep,NaN,NaN,NaN,7.18
1,2024-02-15 21:00:00,Train,Tennis,NaN,NaN,1.00
2,2024-02-16 10:00:00,Eat,NaN,798.0,NaN,NaN
3,2024-02-16 10:02:00,Weight,NaN,NaN,56.2,NaN
4,2024-02-16 13:46:00,Train,Tennis,NaN,NaN,1.00
5,2024-02-16 14:09:00,Sleep,NaN,NaN,NaN,8.25
6,2024-02-16 14:36:00,Eat,NaN,405.0,NaN,NaN
7,2024-02-16 15:33:00,Eat,NaN,135.0,NaN,NaN


In [120]:
date = pd.Timestamp.now()

In [121]:
delta_date = pd.to_timedelta(f'{1 * 7} days')
period_date = date - delta_date

In [122]:
df[df['Date'].dt.date >= period_date.date()]

,Date,Type,Info,Kk,Kg,Hours
0,2024-02-15 21:00:00,Sleep,NaN,NaN,NaN,7.18
1,2024-02-15 21:00:00,Train,Tennis,NaN,NaN,1.00
2,2024-02-16 10:00:00,Eat,NaN,798.0,NaN,NaN
3,2024-02-16 10:02:00,Weight,NaN,NaN,56.2,NaN
4,2024-02-16 13:46:00,Train,Tennis,NaN,NaN,1.00
5,2024-02-16 14:09:00,Sleep,NaN,NaN,NaN,8.25
6,2024-02-16 14:36:00,Eat,NaN,405.0,NaN,NaN
7,2024-02-16 15:33:00,Eat,NaN,135.0,NaN,NaN


In [160]:
df_gb = df.groupby(['Type', pd.Grouper(key='Date', freq='D')], as_index=False).agg(
    Kk=('Kk', 'sum'),
    Kg=('Kg', 'first'),
    Hours=('Hours', 'sum'),
).sort_values(by='Date')

df_gb = df_gb.fillna(0)

for column in ['Kk', 'Kg']:
    df_gb[column + '_num'] = df_gb[column] / df_gb[column].max()

idx = df_gb[df_gb['Type'] == 'Sleep'].index
df_gb.loc[idx, 'Hours_num'] = df_gb.loc[idx, 'Hours'] / df_gb.loc[idx, 'Hours'].max()

idx = df_gb[df_gb['Type'] == 'Train'].index
df_gb.loc[idx, 'Hours_num'] = df_gb.loc[idx, 'Hours'] / df_gb.loc[idx, 'Hours'].max()

In [161]:
df_gb

,Type,Date,Kk,Kg,Hours,Kk_num,Kg_num,Hours_num
1,Sleep,2024-02-15,0.0,0.0,7.18,0.0,0.000000,0.870303
3,Train,2024-02-15,0.0,0.0,1.00,0.0,0.000000,1.000000
5,Weight,2024-02-15,0.0,56.0,0.00,0.0,0.996441,NaN
0,Eat,2024-02-16,1338.0,0.0,0.00,1.0,0.000000,NaN
2,Sleep,2024-02-16,0.0,0.0,8.25,0.0,0.000000,1.000000
4,Train,2024-02-16,0.0,0.0,1.00,0.0,0.000000,1.000000
6,Weight,2024-02-16,0.0,56.2,0.00,0.0,1.000000,NaN


In [162]:
import plotly.express as px
import plotly.graph_objects as go

In [185]:
# fig = px.bar(data_frame=df_gb[df_gb['Type'] == 'Sleep'], y='Hours', x='Date')
fig = go.Figure(
    layout=dict(
        xaxis=dict(categoryorder="category descending"),
        yaxis=dict(range=[-0.05, 1.05]),
        scattermode="group",
        legend=dict(groupclick="toggleitem"),
    )
)

fig.add_trace(
    go.Bar(
        x=df_gb[df_gb['Type'] == 'Sleep']['Date'],
        y=df_gb[df_gb['Type'] == 'Sleep']['Hours_num'],
        name="Sleep",
        hovertext=df_gb[df_gb['Type'] == 'Sleep']['Hours'],
        marker_color="darkcyan",
    )
)

fig.add_trace(
    go.Bar(
        x=df_gb[df_gb['Type'] == 'Train']['Date'],
        y=df_gb[df_gb['Type'] == 'Train']['Hours_num'],
        name="Train",
        hovertext=df_gb[df_gb['Type'] == 'Train']['Hours'],
        marker_color="sandybrown",
    )
)

fig.add_trace(
    go.Bar(
        x=df_gb[df_gb['Type'] == 'Eat']['Date'],
        y=df_gb[df_gb['Type'] == 'Eat']['Kk_num'],
        name="Eat",
        hovertext=df_gb[df_gb['Type'] == 'Eat']['Kk'],
        marker_color="rosybrown",
    )
)

fig.add_trace(
    go.Scatter(
        x=df_gb[df_gb['Type'] == 'Weight']['Date'],
        y=df_gb[df_gb['Type'] == 'Weight']['Kg_num'],
        name="Kg",
        hovertext=df_gb[df_gb['Type'] == 'Weight']['Kg'],
        mode='lines+markers',
        marker_color="red",
    )
)

fig.update_xaxes(
    tickformat="%d %b")

fig.show()